In [1]:
import numpy as np 
import pygame 

from formula_functions import *
from animation_functions import *
from intersection_functions import create_spiral, distance
from show_parameters_functions import *

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Global animation variables

""" 

screen_width = 1500
screen_height = 800
FPS = 25

In [13]:
def main_loop(screen_width, screen_height, FPS):
    '''
    Initializations
    '''
    
    pygame.init()
    # font_big = pygame.font.Font(None, 36)
    font_small = pygame.font.Font(None, 26)
    # message_font = pygame.font.Font(None, 20)
    # titles_font = pygame.font.SysFont('arial', 36)
    bg_color = (255, 255, 255)
    line_color = (185, 185, 185) # Color of the coordinate line
    clock = pygame.time.Clock()
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer
    background_surface = pygame.Surface((screen_width, screen_height))
   
    line_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    spiral_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    params_layer = pygame.Surface((screen_width, screen_height),pygame.SRCALPHA)
    

    background_surface.fill(bg_color)
    line_layer.fill((0,0,0,0))
    spiral_layer.fill((0, 0, 0, 0))
    params_layer.fill((0, 0, 0, 0))

    
    '''
    Global variables
    '''
    
    units =20
    length = screen_width/ units
    v_step = 0.02
    w_step = 0.05
    k_step = 0.02
    x_step = 0.02
    
    v = 2
    w = 5
    k = 0
    vert_line_x = 1
    
    const_params_dict = {'v': v, 'w': w, 'k': k, 'x': vert_line_x}
    
    v_additional, w_additional, x_additional, k_additional = 0, 0, 0, 0
    
    var_params_dict = {'v': v_additional, 
                       'w': w_additional, 
                       'k': k_additional, 
                       'x': x_additional}
    
    layers_list = [background_surface, line_layer, spiral_layer, params_layer]
    
    background_surface, markers, screen_objects = create_background(background_surface, screen_width, screen_height, units, length, bg_color, font_small)
    
    x, y = x_transform(0, screen_width, length), y_transform(6, screen_height, length)
    
    center_points = x_transform(-12, screen_width, length), y_transform(0, screen_height, length)

    draw_vertical_line(line_layer, screen_width, screen_height, length, units)
    
    draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v,w=w,k=k)
    
    show_parameters(params_layer, font_small, const_params_dict,
                    var_params_dict)
    
    
    # Atach layers to screen
    blit_layers(win, layers_list, bg_color)
      
    running = True

    while running:
        
        clock.tick(FPS)
        
        udpate_screen = False
      
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        #Check for key combinations
        keys = pygame.key.get_pressed()

        # Left and right movement of the line
        if keys[pygame.K_x] and keys[pygame.K_RIGHT]:

            x_additional += x_step           
            new_x = vert_line_x + x_additional
            line_layer.fill((0,0,0,0))
            draw_vertical_line(line_layer, screen_width, screen_height, length, units, x_axis_value=new_x)

            udpate_screen = True

        elif keys[pygame.K_x] and keys[pygame.K_LEFT]:
            x_additional -= x_step

            new_x = vert_line_x + x_additional
            line_layer.fill((0,0,0,0))
            draw_vertical_line(line_layer, screen_width, screen_height, length, units, x_axis_value=new_x)

            udpate_screen = True

        # Increase and decrease the angluar velocity `w`    
        elif keys[pygame.K_w] and keys[pygame.K_UP]:
            w_additional += w_step

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        elif keys[pygame.K_w] and keys[pygame.K_DOWN]:
            w_additional -= w_step

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        elif keys[pygame.K_k] and keys[pygame.K_UP]:
            k_additional += k_step
            new_k = k + k_additional

            if new_k >= 4:         
                k_additional = 0

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True
        elif keys[pygame.K_k] and keys[pygame.K_DOWN]:
            k_additional -= k_step
            new_k = k + k_additional

            if new_k <= 0:         
                k_additional = 4


            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        elif keys[pygame.K_v] and keys[pygame.K_UP]:
            v_additional += v_step
            new_v = v + v_additional            

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        elif keys[pygame.K_v] and keys[pygame.K_DOWN]:
            v_additional -= v_step
            new_v = v + v_additional

            if new_v <= 5* v_step:
                v_additional += v_step

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True

        elif (keys[pygame.K_LSHIFT] or keys[pygame.K_RSHIFT]) and keys[pygame.K_r]:

            v_additional, w_additional, x_additional, k_additional = 0, 0, 0, 0

            draw_spiral(spiral_layer, screen_width, screen_height,length,units, v=v+v_additional,w= w+w_additional, k= k + k_additional)

            udpate_screen = True


        if udpate_screen:

            var_params_dict = update_var_params_dict(var_params_dict, v_additional, w_additional, x_additional, k_additional)


            show_parameters(params_layer, font_small, const_params_dict,
                            var_params_dict)  


            blit_layers(win, layers_list, bg_color)


        pygame.display.update()
        # pygame.display.flip()
        
    # Exit
    pygame.quit()
    

In [ ]:
main_loop(screen_width, screen_height, FPS)

In [5]:
a = 0
b = 1.5
lim = 40

print(np.linspace(a,b,lim))

unit=(b-a)/(lim-1)
print('unit: ', unit)
ll = []
for i in range(lim):
    ll.append(round(unit*i,8))
# print('ll')
# print(np.array(ll))

[0.         0.03846154 0.07692308 0.11538462 0.15384615 0.19230769
 0.23076923 0.26923077 0.30769231 0.34615385 0.38461538 0.42307692
 0.46153846 0.5        0.53846154 0.57692308 0.61538462 0.65384615
 0.69230769 0.73076923 0.76923077 0.80769231 0.84615385 0.88461538
 0.92307692 0.96153846 1.         1.03846154 1.07692308 1.11538462
 1.15384615 1.19230769 1.23076923 1.26923077 1.30769231 1.34615385
 1.38461538 1.42307692 1.46153846 1.5       ]
unit:  0.038461538461538464


In [6]:
# radian_unit = 2 * np.pi/360
# length = screen_width/ 20

# spiral_coords = []
# distances = []

# v = 0.1
# w = 0.5
# k = 0
# theta_0 = k * np.pi/2
# for i in range(30*361):
#     angle = radian_unit*i
#     t = angle/abs(w)
#     x_spiral = v * t * np.cos(theta_0 + w * t)
#     y_spiral = v * t * np.sin(theta_0 + w * t)
    
#     x_transformed = x_transform(x_spiral, screen_width, length)
#     y_transformed = y_transform(y_spiral, screen_height, length)
    
    
#     spiral_coords.append([x_transformed, y_transformed])
#     if len(spiral_coords) > 1:
#         dist = distance(spiral_coords[-1], spiral_coords[-2])
#         distances.append(int(np.ceil(dist)))
        
# # print(distances, sum(distances))    